# Class 27 - Boolean Networks


In [1]:
import numpy as np

nodes = ['Cell Size', 
         'Cln3', 
         'MBF', 
         'Clb5,6', 
         'Mcm1/SFF', 
         'Swi5', 
         'Sic1', 
         'Clb1,2', 
         'Cdc20&Cdc14', 
         'Cdh1', 
         'Cln1,2', 
         'SBF']

N = len(nodes)

# define the transition matrix 
a = np.zeros([N, N], dtype=int)
a[0,1] = 1
a[1,1] = -1
a[1,2] = 1
a[1,11] = 1
a[2,3] = 1
a[3,4] = 1
a[3,6] = -1
a[3,7] = 1
a[3,9] = -1
a[4,4] = -1
a[4,5] = 1
a[4,7] = 1
a[4,8] = 1
a[5,5] = -1
a[5,6] = 1
a[6,3] = -1
a[6,7] = -1
a[7,2] = -1
a[7,4] = 1
a[7,5] = -1
a[7,6] = -1
a[7,8] = 1
a[7,9] = -1
a[7,11] = -1
a[8,3] = -1
a[8,5] = 1
a[8,6] = 1
a[8,7] = -1
a[8,8] = -1
a[8,9] = 1
a[9,7] = -1
a[10,6] = -1
a[10,9] = -1
a[10,10] = -1
a[11,10] = 1
a = np.matrix(a)

Read the file "Boolean_fixed_points.txt" (a tab-delimited file of ones and zeroes, with a header) and assign to a numpy matrix "fixed_points".  Make a vector of zeroes called "basin_counts" that is the same length as the number of rows of "fixed_points".

In [2]:
fixed_points = None

with open("shared/Boolean_fixed_points.txt", "r") as f:
    f.readline()
    data = f.read();
    data = data.replace("\n", ";\n")
    fixed_points = np.matrix(data[:-2])

print(fixed_points)
print(fixed_points.shape)

[[0 0 0 0 0 0 1 0 0 1 0 0]
 [0 0 1 0 0 0 1 0 0 1 0 0]
 [1 1 1 1 1 1 0 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1]
 [1 1 0 0 1 1 0 1 1 0 1 1]
 [0 0 1 0 0 0 1 0 0 0 0 0]
 [1 1 1 1 1 1 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [1 1 1 0 1 1 0 1 1 0 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
(11, 12)


Define a function `hamming.dist` that gives the hamming distance between two states of the Boolean network (as numpy arrays of ones and zeroes)

In [3]:
def hamming_dist(x1, x2):
    return np.sum(np.abs(x1-x2))

Define a function `evolve` that takes the network from one Boolean vector state to another Boolean vector state

In [4]:
def evolve(state):
    result = (state*a).A1
    
    result[result > 0] = 1
    
    for i in range(len(result)):
        if result[i] == 0:
            result[i] = state[i]
            
    result[result < 0] = 0
    
    return result

Write a function that runs 10,000 simulations of the network. In each simulation, the procedure is:
- create a random binary vector of length 12, and call that vector `state`
- iteratively call "evolve", passing the `state` to evolve and then updating `state` with the return value from `evolve`
- check if `state` changes in the last call to `evolve`; if it does not, then you have reached a fixed point; stop iterating
- compare the state to the rows of `fixed_points`; for the unique row `i` for which you find a match, increment the element in position `i` of `basin_counts`
- print out `basin_counts`

In [6]:
import random

basin_counts = np.zeros(fixed_points.shape[0]+1, dtype=int)

for test in range(10000):
    random_vector = [0]
    while len(random_vector) < 12:
        random_vector.append(random.choice([0, 1]))
        
    state = np.array(random_vector)
    next_state = evolve(state)
        
    while hamming_dist(state, next_state) != 0:
        state = next_state
        next_state = evolve(state)
        
    for i in range(len(fixed_points)):
        if np.array_equal(state, fixed_points[i].A1):
            basin_counts[i] += 1
            break
        
print("Basin counts:", basin_counts)

Basin counts: [8574  543    0  746    0   40    0    5    0   58   34    0]
